## Imports and Functions

In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
    polygons,
)

colors = pc.qualitative.D3
angstrom = "\u212B"

# 5-6 ML Film (XUV): HOMO

# Load Data

In [ ]:
## Load averaged, FFT filtered, K corrected data ##

ddir = r"E:\atully\arpes_data\2022_April\ARPES\C60\k_corrected"
# files = ["FS2andFS4_avgkw_filteredFFT_0.00int.h5"]
files = ["FS2FS4_FFTstreakCor_gkw.h5"]

# This works, but makes dataclass with theta and phi_or_time instead of kx and ky
ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
ad = ARPES_DATA[files[0]]
for k in ["energy", "theta", "phi_or_time"]:
    print(f"{k}.shape = {getattr(ad, k).shape}")
print(f"Data.shape = {ad.data.shape}")

# Analysis

## HS Cuts

In [ ]:
## Initial params ##
slice_dim = "x"
EF = 18.48

slice_val = 0
int_range = 0.1

x_bin = 1
y_bin = 1

# xlim = None
# ylim = None

xlim = (-1.3, 0.18)
ylim = (-2.7 + EF, -1.6 + EF)

In [ ]:
## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)

## Plot data
fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> HOMO: k<sub>y</sub> = {slice_val}",
    xaxis_title="$k_x \; [A^{-1}]$",
    # yaxis_title="$E_K \; [eV]$",
    yaxis_title="$E - E_F \; [eV]$",
    gridlines=False,
    equiv_axes=False,
)

fig.add_trace(go.Heatmap(x=x_2d, y=y_2d - EF, z=d_2d, coloraxis="coloraxis"))

fig.show()

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    title=f"EDC of HOMO",
    xaxis_title="$E - E_F \; [eV]$",
    yaxis_title="Intensity [arb. u]",
    equiv_axes=False,
    gridlines=False,
    height=600,
    width=1000,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=x_2d, y=y_2d, data=d_2d, ylims=(15.8, 17.1), x_range=None
)

# Plot Data
fig.add_trace(go.Scatter(x=y_1d - EF, y=col, name="data", line=dict(color=colors[0])))

fig.show()

In [ ]:
## Fit Data ##

x = y_1d - EF
data = col

offset_type = "constant"

## Offset
# c = np.mean(data)
c = 0
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

## Gaussian
gauss = fitting_functions.make_gaussian(num="A_", amplitude=30, center=-2, sigma=0.5)

## Lorentzians
# lorentz1 = fitting_functions.make_lorentzian(
#     num="A_", amplitude=0.1, center=14.8, sigma=0.2
# )
# lorentz2 = fitting_functions.make_lorentzian(
#     num="B_", amplitude=0.1, center=15.1, sigma=0.5
# )
lorentz1 = fitting_functions.make_lorentzian(num="A_", amplitude=30, center=-2, sigma=3)

full_model = gauss + offset
# full_model = lorentz1 + offset

fit = full_model.fit(data, x=x)

fit.plot()

print(fit.params["iA__fwhm"])
print(fit.params["iA__center"])

In [ ]:
fit.params

## FS Reconstruction

In [ ]:
## Initial params ##
slice_dim = "y"
EF = 18.48

HOMO_center = EF - 2.09

slice_val = 16.8
# slice_val = 16.4
# slice_val = 16.0

int_range = 0.1

x_bin = 1
y_bin = 1

In [ ]:
title = f"C<sub>60</sub> HOMO ({np.round(slice_val-HOMO_center,1)} eV)"
yaxis_title = f"k<sub>y</sub> [{angstrom}<sup>-1</sup>]"
xaxis_title = f"k<sub>x</sub> [{angstrom}<sup>-1</sup>]"

In [ ]:
## Limit Dataset ##

xlim = (-0.52, 0.52)
ylim = (-1.25, 0.15)

## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)


fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    dtick_y=0.5,
    height=600,
    width=600,
)

## Plot Data
fig.add_trace(
    go.Heatmap(
        x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
    )
)


hexagon = polygons.gen_polygon(6, 0.42, translation=(0, -0.375 * 2), rotation=30)
fig = polygons.plot_polygon(
    hexagon, color="yellow", fig=fig, show=False, dash=True, dash_width=3
)

if xlim is not None:
    fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

fig.update_coloraxes(colorscale="Blues", reversescale=False)

fig.show()

# fig.write_image(
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.8eV_rawdata.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.4eV_rawdata.png"
#     r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.0eV_rawdata.png"
# )

In [ ]:
## Subset Data for Rotation ##

xlim = (-0.52, 0.52)
# xlim = (
#     -0.21,
#     0.52,
# )  # limit for BZ_small_avg; basically limiting original data to the smallest region necessary to reconstruct full BZ

ylim = (-0.45, 0.15)

## Get data
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)

## Plot data
fig = tr_functions.thesis_fig(
    title=f"C<sub>60</sub> HOMO: {slice_val} eV",
    xaxis_title="$k_x \; [A^{-1}]$",
    yaxis_title="$k_y \; [A^{-1}]$",
)

fig.add_trace(
    go.Heatmap(
        x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
    )
)

hexagon = polygons.gen_polygon(6, 0.42, rotation=30)
fig = polygons.plot_polygon(
    hexagon, color="firebrick", fig=fig, show=False, dash=True, dash_width=3
)

if xlim is not None:
    fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

fig.show(renderer="svg")

In [ ]:
## Rotate Data ##

In [ ]:
x, y, z = x_2d, y_2d, analysis_functions.norm_data(d_2d)

coords = tr_functions.x_y_to_coords(x, y)

rotated_coords = tr_functions.rotate_2d_array(coords, 120, (0, 0))
rotated_coords_2 = tr_functions.rotate_2d_array(coords, 240, (0, 0))

nx, ny, nd = tr_functions.interpolate(rotated_coords, z)
nx_2, ny_2, nd_2 = tr_functions.interpolate(rotated_coords_2, z)

In [ ]:
# ## Plot Rotated Data to Check ##

# x1, y1 = x_2d, y_2d
# x2, y2 = nx, ny

# new_x = np.linspace(min(min(x1), min(x2)), max(max(x1), max(x2)), 1000)
# new_y = np.linspace(min(min(y1), min(y2)), max(max(y1), max(y2)), 1000)

# fig = tr_functions.thesis_fig(
#     title=f"C<sub>60</sub> HOMO: {slice_val} eV",
#     xaxis_title="$k_x \; [A^{-1}]$",
#     yaxis_title="$k_y \; [A^{-1}]$",
# )

# fig.add_trace(
#     go.Heatmap(
#         x=new_x,
#         y=new_y,
#         z=analysis_functions.norm_data(nd),
#         coloraxis="coloraxis",
#     )
# )

# fig.add_trace(
#     go.Heatmap(
#         x=x_2d,
#         y=y_2d,
#         z=analysis_functions.norm_data(d_2d),
#         coloraxis="coloraxis",
#     )
# )

# hexagon = polygons.gen_polygon(6, 0.42, rotation=30)
# fig = polygons.plot_polygon(
#     hexagon, color="firebrick", fig=fig, show=False, dash=True, dash_width=3
# )


# fig.show()

In [ ]:
## Stitch and/or average original dataset with 1st rotated dataset ##

x1, y1, dataslice1 = x_2d, y_2d, analysis_functions.norm_data(d_2d)
x2, y2, dataslice2 = nx, ny, nd

new_x, new_y, new_data = tr_functions.stitch_and_avg(
    x1,
    y1,
    dataslice1,
    x2,
    y2,
    dataslice2,
    no_avg=True,
)

In [ ]:
# Stitch and/or average new dataset with 2nd rotated dataset ##

x1, y1, dataslice1 = new_x, new_y, new_data
x2, y2, dataslice2 = nx_2, ny_2, nd_2

new_x, new_y, new_data = tr_functions.stitch_and_avg(
    x1,
    y1,
    dataslice1,
    x2,
    y2,
    dataslice2,
    no_avg=True,
)

In [ ]:
## Plot Reconstructed 1st BZ ##

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    equiv_axes=True,
    height=600,
    width=600,
)

fig.add_trace(
    go.Heatmap(
        x=new_x,
        y=new_y,
        z=analysis_functions.norm_data(new_data),
        coloraxis="coloraxis",
    )
)

hexagon = polygons.gen_polygon(6, 0.42, rotation=30)
fig = polygons.plot_polygon(
    hexagon, color="yellow", fig=fig, show=False, dash=True, dash_width=3
)


fig.show()

# fig.write_image(
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.8eV_BZ1_noavg.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.4eV_BZ1_noavg.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.0eV_BZ1_noavg.png"
# )

# fig.write_image(
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.8eV_BZ1_averaged.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.4eV_BZ1_averaged.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.0eV_BZ1_averaged.png"
# )

# fig.write_image(
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.8eV_BZ1_small_averaged.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.4eV_BZ1_small_averaged.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.0eV_BZ1_small_averaged.png"
# )

In [ ]:
# ## Compare to full 2nd BZ ##

# xlim = (-0.52, 0.52)
# ylim = (-1.25, -0.32)

# ## Get data
# x_2d, y_2d, d_2d = tr_functions.slice_datacube(
#     ad,
#     slice_dim,
#     slice_val,
#     int_range,
#     xlim,
#     ylim,
#     x_bin,
#     y_bin,
#     norm_data=True,
#     plot_data=False,
# )

# ## Plot data
# fig = tr_functions.thesis_fig(
#     title=f"C<sub>60</sub> HOMO: {slice_val} eV",
#     xaxis_title="$k_x \; [A^{-1}]$",
#     yaxis_title="$k_y \; [A^{-1}]$",
# )

# fig.add_trace(
#     go.Heatmap(
#         x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
#     )
# )

# hexagon = polygons.gen_polygon(6, 0.42, rotation=30, translation=(0, -0.36 * 2))
# fig = polygons.plot_polygon(
#     hexagon, color="firebrick", fig=fig, show=False, dash=True, dash_width=3
# )

# if xlim is not None:
#     fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

# fig.show(renderer="svg")

# # fig.write_image(
# #     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.8eV_BZ2_rawdata.png"
# #     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.4eV_BZ2_rawdata.png"
# #     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.0eV_BZ2_rawdata.png"
# # )

In [ ]:
# ## Stitch and Average 1st and 2nd BZ back together ##

# x1, y1, dataslice1 = x_2d, y_2d, analysis_functions.norm_data(d_2d)
# x2, y2, dataslice2 = new_x, new_y, analysis_functions.norm_data(new_data)

# full_x, full_y, full_data = tr_functions.stitch_and_avg(
#     x1, y1, dataslice1, x2, y2, dataslice2
# )

In [ ]:
## Plot Data ##

xlim = (-0.52, 0.52)
ylim = (-1.25, 0.15)

## Get original data again
x_2d, y_2d, d_2d = tr_functions.slice_datacube(
    ad,
    slice_dim,
    slice_val,
    int_range,
    xlim,
    ylim,
    x_bin,
    y_bin,
    norm_data=True,
    plot_data=False,
)


fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    gridlines=False,
    dtick_y=0.5,
    height=600,
    width=600,
)

## Plot two datasets on top of each other
fig.add_trace(
    go.Heatmap(
        x=new_x,
        y=new_y,
        z=analysis_functions.norm_data(new_data),
        coloraxis="coloraxis",
    )
)


fig.add_trace(
    go.Heatmap(
        x=x_2d, y=y_2d, z=analysis_functions.norm_data(d_2d), coloraxis="coloraxis"
    )
)

# ## Plot stitched and averaged dataset
# fig.add_trace(
#     go.Heatmap(
#         x=full_x,
#         y=full_y,
#         z=analysis_functions.norm_data(full_data),
#         coloraxis="coloraxis",
#     )
# )


hexagon = polygons.gen_polygon(6, 0.42, rotation=30)
fig = polygons.plot_polygon(
    hexagon, color="yellow", fig=fig, show=False, dash=True, dash_width=3
)

if xlim is not None:
    fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

fig.update_coloraxes(colorscale="Blues", reversescale=False)

fig.show()

# fig.write_image(
#     r"C:\Users\atully\OneDrive\Physics.UBC\PhD\2DGC Figs\HOMO_16.8eV_full.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\PhD\2DGC Figs\HOMO_16.4eV_full.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\PhD\2DGC Figs\HOMO_16.0eV_full.png"
# )

# fig.write_image(
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.8eV_full_averaged.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.4eV_full_averaged.png"
#     # r"C:\Users\atully\OneDrive\Physics.UBC\TR-ARPES\Data\HOMO_16.0eV_full_averaged.png"
# )